SPRINT ENGENHARIA DE DADOS - ENZO RIBEIRO

1. Qual é a descrição do problema?

2. Você tem premissas ou hipóteses sobre o problema? Quais?

3. Que restrições ou condições foram impostas para selecionar os dados?

4. Descreva o seu dataset (atributos, imagens, anotações, etc).

1. Esse MVP se baseia em dados do Mercado de Luxo de Cosméticos no Brasil. O problema principal é responder 6 perguntas gerenciais sobre os dados contidos no DataSet que irão ajudar a entender o que está acontecendo nesse mercado:

- Qual foi o ano que teve maior evolução de vendas nos ultimos anos?
- Qual canal que mais cresceu nesse ano?
- Houve mudança no preço praticado entre os canais nos últimos anos?
- Qual a categoria com maior peso no mercado em 2021 e qual categoria com maior peso em 2023? O peso mudou entre a pandemia e atualmente?
- Qual a subcategoria com menor preço medio de venda?
- Qual mês com maior venda em 2023?

Para isso, irei montar um Data Lakehouse com camadas Bronze, Silver e Gold, iniciando na coleta, modelagem, carga e análise desses dados;

2. Eu tenho a hipótese que o mercado Online está bem desacelerado em 2023 versus os anos anteriores que ainda tinham resquícios da pandemia e que atualmente os consumidores procuram mais a Sephora (que se enquadra no canal Other B&M) para comprar produtos de beleza.

3. Esses dados possuem confidencialidade, logo foi necessário tirar colunas que possuiam um maior número de detalhe e manipular, antes da importação, os valores de venda, se retratando, então, de dados fictícios.

4. Dataset:
- 'Channel': canal de vendas, pode ser físico (Department Stores ou Other B&M) ou Online
- 'Company': nome da empresa
- 'Category': categoria do produto de beleza (fragrâncias, maquiagem ou skincare)
- 'Subcategory': subcategoria com detalhes adicionais
- 'Year', 'Month': atributos referentes a data de venda
- 'U', '$': venda em unidades e valor (representado em reais)

CAMADA BRONZE - COLETA

Este script demonstra um exemplo simples da camada Bronze em um Data Lakehouse, utilizando um arquivo CSV hospedado no GitHub como fonte de dados brutos. Isso porque os dados utilizados são da minha rotina profissional e não podem ser acessados de outra forma.

O objetivo principal é baixar os dados brutos e torná-los disponíveis para processamento posterior nas camadas Silver e Gold.

Para isso iremos salvar esse dataframe como uma tabela chamada bronze_mercado dentro do bronze_database

In [0]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark.sql.types import *

#Importando dados de acidentes

url = "https://raw.githubusercontent.com/ribeiroenzo/PUC-MBA/main/Base_Mercado2.csv"
urllib.request.urlretrieve(url, "/tmp/Base_Mercado2.csv")
dbutils.fs.mv("file:/tmp/Base_Mercado2.csv","/mnt/dados/Base_Mercado2.csv")
mercado_df = spark.read.format("csv").option("header", True).option("sep",";").option("multiLine", True).load("/mnt/dados/Base_Mercado2.csv")

#Mostra o dataframe
display(mercado_df)

#Cria o banco de dados se ele não existe
spark.sql(f"CREATE DATABASE IF NOT EXISTS bronze_database")

#Popula a tabela no banco de dados
mercado_df.write.mode("overwrite").format("delta").option("overwriteSchema","true").saveAsTable("bronze_database.bronze_mercado")


Channel,Company,Category,Subcategory,Year,Month,U,$
B&M,Estée Lauder Brasil,Make Up,Mascaras,2022,Nov,28260,3023575
B&M,Other exclusives,Make Up,Blush,2023,Dec,20144,3125859
B&M,Other exclusives,Make Up,Blush,2023,Nov,17364,2529059
B&M,Other exclusives,Make Up,Foundations,2021,Nov,14385,1981918
Online,Other exclusives,Make Up,Blush,2023,Nov,14042,1868092
Online,Other exclusives,Make Up,Foundations,2021,Nov,10480,1379653
B&M,Other exclusives,Make Up,Lip Gloss,2021,Nov,8978,631928
B&M,Estée Lauder Brasil,Make Up,Mascaras,2021,Nov,13511,1988003
B&M,Other exclusives,Make Up,Blush,2023,Oct,8719,1411408
Online,Other exclusives,Make Up,Lip Gloss,2021,Nov,7376,513015
